In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
print(stocks.tail())

    Ticker
496    YUM
497    ZBH
498   ZBRA
499   ZION
500    ZTS


In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 95530963,
 'calculationPrice': 'iexlasttrade',
 'change': -5.18,
 'changePercent': -0.03317,
 'close': 0,
 'closeSource': 'fciolafi',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 156.72,
 'iexCloseTime': 1711354965147,
 'iexLastUpdated': 1713121758281,
 'iexMarketPercent': 0.016990941930153415,
 'iexOpen': 159.12,
 'iexOpenTime': 1725926354398,
 'iexRealtimePrice': 154.13,
 'iexRealtimeSize': 1,
 'iexVolume': 2191926,
 'lastTradeTime': 1684756097321,
 'latestPrice': 157.52,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'May 9, 2022',
 'latestUpdate': 1680045691418,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,
 'marketCap

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']
price, market_cap

(157.52, 2574478639735)

In [6]:
columns = ['Ticker', 'Stock Price', 'Market Cap.', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns = columns)
final_df

,Ticker,Stock Price,Market Cap.,Number of Shares to Buy


In [7]:
final_df.append(
    pd.Series([symbol, price, market_cap, 'NaN' ], index=columns), ignore_index=True
)
final_df

/tmp/ipykernel_857/3831238531.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df.append(


,Ticker,Stock Price,Market Cap.,Number of Shares to Buy


In [8]:
# BATCH API CALL
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_df = pd.DataFrame(columns = columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
                                        pd.Series([symbol,
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['marketCap'],
                                                   'NaN'],
                                                  index = columns),
                                        ignore_index = True)


/tmp/ipykernel_857/2602819213.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_857/2602819213.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_857/2602819213.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_857/2602819213.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_857/2602819213.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(


In [10]:
final_df

,Ticker,Stock Price,Market Cap.,Number of Shares to Buy
0,A,115.65,34898776716,NaN
1,AAL,17.01,11111649025,NaN
2,AAP,204.75,12490596038,NaN
3,AAPL,156.81,2483539867292,NaN
4,ABBV,151.18,275459684018,NaN
...,...,...,...,...
496,YUM,115,32881134200,NaN
497,ZBH,116,25184999091,NaN
498,ZBRA,336.52,17336279669,NaN
499,ZION,57.31,8410691763,NaN


In [11]:
portfolio_size = input('Enter the value of your portfolio: ')
try:
    portfolio_size = float(portfolio_size)
except ValueError:
    print('Error: enter a int or float.')
print(portfolio_size)

10000000.0


In [12]:
position_size = portfolio_size/len(final_df.index)
for i in range(0, len(final_df.index)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Stock Price'])
final_df

,Ticker,Stock Price,Market Cap.,Number of Shares to Buy
0,A,115.65,34898776716,172
1,AAL,17.01,11111649025,1173
2,AAP,204.75,12490596038,97
3,AAPL,156.81,2483539867292,127
4,ABBV,151.18,275459684018,132
...,...,...,...,...
496,YUM,115,32881134200,173
497,ZBH,116,25184999091,172
498,ZBRA,336.52,17336279669,59
499,ZION,57.31,8410691763,348


In [13]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, 'Recommended Trades', index=False)

In [14]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1

    }
)
int_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,

    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1

    }
)

In [15]:
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Cap.', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to buy', int_format)

writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
writer.save()